In [8]:
import requests
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
offense_type = "?offense_description=BURGLARY- AGGRAVATED"

endpoint = f'https://data.nashville.gov/resource/2u6v-ujjs.json{offense_type}'
response = requests.get(endpoint)

In [10]:
response

<Response [200]>

In [11]:
response.text

'[{"primary_key":"20171099391_21","incident_number":"20171099391","report_type":"D","report_type_description":"DISPATCHED","incident_status_code":"A","incident_status_description":"CLEARED BY ARREST","investigation_status":"Closed","incident_occurred":"2017-12-16T18:30:00.000","incident_reported":"2017-12-16T22:44:00.000","incident_location":"1101 1101","latitude":"36.178","longitude":"-86.586","zip_code":"37076","rpa":"9539","zone":"523","location_code":"22","location_description":"RESIDENCE, HOME","offense_number":"2","offense_nibrs":"220","offense_description":"BURGLARY- AGGRAVATED","weapon_primary":"09","weapon_description":"PERSONAL (HANDS)","victim_number":"1","domestic_related":true,"victim_type":"I","victim_description":"INDIVIDUAL (18 AND OVER)","victim_gender":"F","victim_race":"B","victim_ethnicity":"Non-Hispanic","victim_county_resident":"RESIDENT","mapped_location":{"type":"Point","coordinates":[-86.586,36.178]}}\n,{"primary_key":"20190582775_11","incident_number":"2019058

In [12]:
api_data = response.json()

In [13]:
api_data_df = pd.DataFrame(api_data)

In [14]:

incident_occured_dt = pd.to_datetime(api_data_df['incident_occurred'])
incident_occured_dt

0     2017-12-16 18:30:00
1     2019-08-05 23:08:00
2     2019-05-24 15:00:00
3     2016-03-12 02:20:00
4     2021-02-06 14:41:00
              ...        
995   2021-09-11 00:25:00
996   2021-08-29 04:00:00
997   2021-10-26 12:00:00
998   2022-01-01 12:00:00
999   2021-07-10 16:50:00
Name: incident_occurred, Length: 1000, dtype: datetime64[ns]

In [15]:
time_1 = '2021-01-01 00:00:00'
time_2 = '2021-09-30 23:59:59'
boolean = (incident_occured_dt >= time_1) & (incident_occured_dt <= time_2)

agg_burglary_data_df = api_data_df[boolean]

In [16]:
agg_burglary_data_df

,primary_key,incident_number,report_type,report_type_description,incident_status_code,incident_status_description,investigation_status,incident_occurred,incident_reported,incident_location,...,weapon_description,victim_number,domestic_related,victim_type,victim_description,victim_gender,victim_race,victim_ethnicity,victim_county_resident,mapped_location
4,20210073684_11,20210073684,D,DISPATCHED,A,CLEARED BY ARREST,Closed,2021-02-06T14:41:00.000,2021-02-06T17:35:00.000,706 706,...,CLUB,1,False,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.73, 36.12]}"
8,20210077341_11,20210077341,S,SUSPECT,A,CLEARED BY ARREST,Closed,2021-01-23T08:49:00.000,2021-02-08T12:25:00.000,3020 3020,...,NONE,1,True,I,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.818, 36...."
9,20210249540_11,20210249540,D,DISPATCHED,O,OPEN,Open,2021-05-04T23:45:00.000,2021-05-05T00:45:00.000,UNIVERSITY CT,...,PERSONAL (HANDS),1,False,I,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.77, 36.15]}"
10,20210248840_11,20210248840,D,DISPATCHED,O,OPEN,Open,2021-05-04T12:06:00.000,2021-05-04T14:33:00.000,BONNAFAIR DR,...,PERSONAL (HANDS),1,False,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.63, 36.2]}"
11,20210244919_11,20210244919,D,DISPATCHED,O,OPEN,Open,2021-05-01T23:00:00.000,2021-05-02T12:40:00.000,8TH AVE S,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,NON RESIDENT,"{'type': 'Point', 'coordinates': [-86.78, 36.14]}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,20220026083_11,20220026083,D,DISPATCHED,A,CLEARED BY ARREST,Closed,2021-01-18T21:46:00.000,2022-01-18T22:34:00.000,701 701,...,HANDGUN,1,False,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.696, 36...."
993,20210481395_11,20210481395,D,DISPATCHED,O,OPEN,Open,2021-09-12T18:00:00.000,2021-09-13T03:32:00.000,SPENCE LN,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.72, 36.14]}"
995,20210478574_11,20210478574,D,DISPATCHED,A,CLEARED BY ARREST,Closed,2021-09-11T00:25:00.000,2021-09-11T03:09:00.000,618 618,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.732, 36...."
996,20210479042_11,20210479042,D,DISPATCHED,O,OPEN,Open,2021-08-29T04:00:00.000,2021-09-11T11:19:00.000,LESTER AVE,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),F,W,Unknown,UNKNOWN,"{'type': 'Point', 'coordinates': [-86.75, 36.14]}"


In [17]:
type(agg_burglary_data_df['mapped_location'])

pandas.core.series.Series

In [22]:
coordinates = agg_burglary_data_df['mapped_location'].get('coordinates', default = None)